In [ ]:
# importing differnt packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# reading the input data
df=pd.read_csv("C:/Users/vrunda/job-hackothon/Second-competiton/train_s3TEQDk.csv")

In [ ]:
df

In [ ]:
# checking data info
df.info()

In [ ]:
# finding nan columns
df.isna().sum()

In [ ]:
# droping id column
df=df.drop('ID',axis=1)

In [ ]:
# feature enginnering , applying label encoding for categorical columns and creating boolean masking of it

for label,content in df.items():
    if not pd.api.types.is_numeric_dtype(content):
        # add binary column wheter sample has missing values or not
        df[label+'_is_missing']=pd.isnull(content)
        #turn categories into numbers and add 1
        df[label]=pd.Categorical(content).codes+1

In [ ]:
df

In [ ]:
# dropping unnecessary columns
df=df.drop(['Gender_is_missing','Region_Code_is_missing','Occupation_is_missing','Channel_Code_is_missing','Is_Active_is_missing'],axis=1)

In [ ]:
# splitting the data into dependent and independant columns
x=df.drop('Is_Lead',axis=1)
y=df['Is_Lead']

In [ ]:
# splitting the data for training and testing
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=142)

In [ ]:
# reading the test data
test=pd.read_csv("C:/Users/vrunda/job-hackothon/Second-competiton/test_mSzZ8RL.csv")

In [ ]:
# creating empty dataframe using id and is_lead columns
sub=pd.DataFrame(columns=['ID','Is_Lead'])

In [ ]:
# filling id values in new dataframe
sub['ID']=test['ID']

In [ ]:
# dropping id column from test data
test=test.drop('ID',axis=1)

In [ ]:
# applying feature enginnering technique for test data
for label,content in test.items():
    if not pd.api.types.is_numeric_dtype(content):
        # add binary column wheter sample has missing values or not
        test[label+'_is_missing']=pd.isnull(content)
        #turn categories into numbers and add 1
        test[label]=pd.Categorical(content).codes+1

In [ ]:
# removing unwanted columns form the data
test=test.drop(['Gender_is_missing','Region_Code_is_missing','Occupation_is_missing','Channel_Code_is_missing','Is_Active_is_missing'],axis=1)

In [ ]:
# light gbm algorithm for model training
NUM_OF_BOOST_ROUND=10000

params={'metric':'auc','seed':'2021','n_estimators':NUM_OF_BOOST_ROUND}
import lightgbm as lgb
clf=lgb.LGBMClassifier(**params)

In [ ]:
# fitting the data in the model
clf.fit(x_train,y_train,eval_set=(x_test, y_test), early_stopping_rounds=100, verbose=-1)

In [ ]:
# predictions on test data
ypred_lgb=clf.predict_proba(x_test)[:,1]

In [ ]:
# final predictions
lgm_results=clf.predict_proba(test_final)[:,1]

In [ ]:
# validating model performance by auc curve
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,y_pred_lgb)

In [ ]:
# filling predictions into dataframe
sub2['Is_Lead']=lgm_results

In [ ]:
# saving the dataframe results in csv format
sub2.to_csv("my_submission.csv",index=False)